In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# stuff needed for google colab
CURR_DIR = "/content/drive/My Drive/Grad School Fall/CS 573/Project/colab_notebooks"
DATA_FILE = "profiles.csv"

In [3]:
import pandas as pd
import numpy as np
import os
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import sys
from sklearn.feature_extraction.text import CountVectorizer
from scipy import sparse
import string
np.set_printoptions(threshold=1000)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
profiles = pd.read_csv(os.path.join(CURR_DIR, DATA_FILE))
print(profiles)

       age  ...     status
0       22  ...     single
1       35  ...     single
2       38  ...  available
3       23  ...     single
4       29  ...     single
...    ...  ...        ...
59941   59  ...     single
59942   24  ...     single
59943   42  ...     single
59944   27  ...     single
59945   39  ...     single

[59946 rows x 31 columns]


In [5]:
train_df = profiles.sample(10000, random_state=42)

In [6]:
# def parseEssay(df):
#   essay_list = df['essay0'].to_list()

#   # parses out the NaN's and leaves them empty
#   print("parsing out nans")
#   for i in range(len(essay_list)):
#     if not isinstance(essay_list[i], str):
#       essay_list[i] = ""

#   return essay_list

def parseEssay(df):
  essay_list = df['essay0'].to_list()

  essay_list1 = df['essay1'].to_list()
  essay_list2 = df['essay2'].to_list()

  # parses out the NaN's and leaves them empty
  # jesus christ look at this garbage code
  print("parsing out nans")
  for i in range(len(essay_list)):
    if not isinstance(essay_list[i], str):
      if not isinstance(essay_list1[i], str):
        if not isinstance(essay_list1[i], str):
          essay_list[i] = ""
        else:
          essay_list[i] = essay_list2[i]
      else:
        essay_list[i] = essay_list1[i]

  return essay_list

preclean_train_essays = parseEssay(train_df)

parsing out nans


In [10]:
table = str.maketrans(dict.fromkeys(string.punctuation))
# keep the ' so that we can have words like "i'm", "can't", etc
del table[39]

# print(table)

def cleanLine(line, punc_table):
  # remove <br />
  # print(line)
  line = line.replace("<br />", " ")
  line = line.replace("\n", " ")
  line = line.lower()
  line = line.translate(punc_table)
  line = ' '.join(line.split())

  return line


def cleanEssays(data):
  table = str.maketrans(dict.fromkeys(string.punctuation))
  for i in range(len(data)):
    data[i] = cleanLine(data[i], table)
    # break

  return data

# print(train_essays[4])
# train_essays = cleanEssays(preclean_train_essays)
print(preclean_train_essays[4])
line = cleanLine(preclean_train_essays[4], table)
print(line)


i come to san francisco by way of new york, where i was born,
raised and went to school. i think this accounts for my
occasionally blunt, no bullshit attitude, although the bay area has
been hard at work mellowing me out. (some say it's even been
successful)<br />
<br />
i've been working in the games industry, in the east bay, as a
producer, designer and occasional contributor of polygonal art for
the past six years. recent projects involved me working with a
bunch of neat folks to make the games where you click buttons on
plastic guitar thingies and pretend to play music. it was great fun
working with my team of bright, super passionate music loving
people, and i learned a lot. but the 14-16 work days eventually
took their toll, so now i work saner hours making educational games
for 3-8 year-olds in emeryville. it's humbling, really, to see all
your design assumptions blown out of the water by a 3 year
old.<br />
<br />
i'm a big fan of city walking. previous to coming to sf, i worke

In [12]:
def createWordVectors(train_essays):
  print("encoding essays")
  print(train_essays)
  vectorizer = CountVectorizer()
  vectorizer.fit(train_essays)

  word_vecs = vectorizer.transform(train_essays)

  return word_vecs, vectorizer

# train_essays = parseEssay(train_df)
word_vecs, vectorizer = createWordVectors(train_essays)

X = word_vecs.toarray()

# for printing out the actual vocab encoding vec in case you want it
# print(vectorizer.vocabulary_)
# encoding document
print(X.shape)
print(len(vectorizer.vocabulary_))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(10000, 39170)
39170


In [13]:
def load_labels(df):
  df['sex'].mask(df['sex'] == 'm', 0, inplace=True)
  df['sex'].mask(df['sex'] == 'f', 1, inplace=True)

  labels = df['sex'].to_numpy()

  return labels

labels = load_labels(train_df)

# Convert to float32's so that they play nicely

In [14]:
X = np.asarray(X).astype('float32')
Y = np.asarray(labels).astype('float32')

# Do some neural net shenannigans

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import KFold
import statistics 

In [16]:
def createModel():
  model = Sequential()
  model.add(Dense(256, input_dim=X.shape[1], activation='relu'))
  model.add(Dense(12, activation='relu'))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  # compile the keras model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model

temp_model = createModel()
temp_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               10027776  
_________________________________________________________________
dense_1 (Dense)              (None, 12)                3084      
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 104       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 10,030,973
Trainable params: 10,030,973
Non-trainable params: 0
_________________________________________________________________


In [16]:
n_split = 10

best_acc = 0
best_model = None

accuracies = []

for train_index,test_index in KFold(n_split).split(X):
  x_train,x_test = X[train_index],X[test_index]
  y_train,y_test = Y[train_index],Y[test_index]

  print(x_train.shape)
  print(y_train.shape)
  model = createModel()

  model.fit(x_train, y_train,epochs=10)
  
  _, test_acc = model.evaluate(x_test,y_test)
  
  accuracies.append(test_acc)

  # break

avg_acc = statistics.mean(accuracies)
print('Test set accuracy: %.2f' % avg_acc)
print(accuracies)

(9000, 39170)
(9000,)
Epoch 1/10
282/282 [==============================] - 1s 4ms/step - loss: 0.6365 - accuracy: 0.6357
Epoch 2/10
282/282 [==============================] - 1s 4ms/step - loss: 0.4417 - accuracy: 0.7836
Epoch 3/10
282/282 [==============================] - 1s 4ms/step - loss: 0.2358 - accuracy: 0.8977
Epoch 4/10
282/282 [==============================] - 1s 4ms/step - loss: 0.1171 - accuracy: 0.9492
Epoch 5/10
282/282 [==============================] - 1s 4ms/step - loss: 0.0722 - accuracy: 0.9633
Epoch 6/10
282/282 [==============================] - 1s 4ms/step - loss: 0.0491 - accuracy: 0.9733
Epoch 7/10
282/282 [==============================] - 1s 4ms/step - loss: 0.0450 - accuracy: 0.9727
Epoch 8/10
282/282 [==============================] - 1s 4ms/step - loss: 0.0424 - accuracy: 0.9741
Epoch 9/10
282/282 [==============================] - 1s 4ms/step - loss: 0.0426 - accuracy: 0.9744
Epoch 10/10
32/32 [==============================] - 0s 3ms/step - loss: 2.015

In [15]:
print(accuracies)

[0.6589999794960022]
